In [1]:
import pandas as pd
import gtfs_kit as gk


In [2]:
feed = gk.feed.read_feed('../general_data_analytics.zip',dist_units="km")

### Collecting a single week data

In [ ]:
### Select trips between Mon 02-Oct-2023 and 06-Oct-2023
